# Capstone Project - Data Cleaning

Objective: Using the given dataset, produce the following insights:
 - 1, The best performing product
 - 2, Information on user demographics
 - 3, Whether sales are seasonal
 - 4, Justification of marketing spends in the regions UK South East, UK North East, UK North, and UK South
 - 5, CPA (cost per acquisition) per region against the expected average (available at https://mystaticwebsite-3.s3.amazonaws.com/index.html)

Created By: Solomon Williams

Creation Date: 04/11/2022

##### Last Updated

In [1]:
from datetime import datetime, timedelta

print(f"Last Updated: {datetime.now().strftime('%d/%m/%Y %H:%M:%S')}")

Last Updated: 08/11/2022 15:31:50


### Importing Packages

In [2]:
import pandas as pd

### Data Gathering and Cleaning

In [3]:
df = pd.read_csv('test-data.csv')
print(df.head())

   order_id     product  quantity order_month   age      uk_region  cpa
0       1.0  toothbrush       1.0      Oct-21  21.0  UK North East  2.0
1       2.0        toys       2.0      Oct-21  35.0  UK South East  3.0
2       3.0  toothbrush       1.0      Oct-21  24.0       UK North  2.0
3       4.0  toothbrush       1.0      Dec-21  21.0  UK North East  4.0
4       5.0  toothbrush       2.0      Oct-21  20.0       UK North  5.0


In [4]:
print(df.info())
print()
print(df.describe())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51 entries, 0 to 50
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   order_id     50 non-null     float64
 1   product      50 non-null     object 
 2   quantity     50 non-null     float64
 3   order_month  50 non-null     object 
 4   age          50 non-null     float64
 5   uk_region    50 non-null     object 
 6   cpa          50 non-null     float64
dtypes: float64(4), object(3)
memory usage: 2.9+ KB
None

       order_id   quantity        age        cpa
count  50.00000  50.000000  50.000000  50.000000
mean   25.50000   1.320000  30.840000   3.180000
std    14.57738   0.586933  20.769247   1.955526
min     1.00000   1.000000 -59.000000   1.000000
25%    13.25000   1.000000  21.000000   2.000000
50%    25.50000   1.000000  31.000000   3.000000
75%    37.75000   1.750000  42.500000   3.000000
max    50.00000   3.000000  60.000000  10.000000


Immediately I can see that the data types are acceptable, though for readability I will set all float values to read as integers. (The decimalisation seems to vary depending on the software used)

In [5]:
pd.options.display.float_format = '{:,.0f}'.format
df.head()

,order_id,product,quantity,order_month,age,uk_region,cpa
0,1,toothbrush,1,Oct-21,21,UK North East,2
1,2,toys,2,Oct-21,35,UK South East,3
2,3,toothbrush,1,Oct-21,24,UK North,2
3,4,toothbrush,1,Dec-21,21,UK North East,4
4,5,toothbrush,2,Oct-21,20,UK North,5


The minimum value for age is negative, which is clearly an input error that I will assume is the correct age, just accidentally made negative. We can correct this by converting it to an absolute value.

In [6]:
df['age'] = df['age'].abs()

df['age'].describe()

count   50
mean    34
std     14
min     20
25%     21
50%     32
75%     49
max     60
Name: age, dtype: float64

In [7]:
print(df.isna().sum())
print()
print(df[df.isna().any(axis=1)])

order_id       1
product        1
quantity       1
order_month    1
age            1
uk_region      1
cpa            1
dtype: int64

    order_id product  quantity order_month  age uk_region  cpa
50       NaN     NaN       NaN         NaN  NaN       NaN  NaN


Each column seems to have an empty row at the end, with only N/a values. This is easily corrected:

In [8]:
df = df.dropna()
print(df.isna().sum())

order_id       0
product        0
quantity       0
order_month    0
age            0
uk_region      0
cpa            0
dtype: int64


In terms of gathering seasonal data, only the month is relevant. As such, I will add columns for month and year independently and drop the order_month column.

In [9]:
df['month'] = df['order_month'].str[:3]
df['year'] = '20' + df['order_month'].str[-2:]

del df['order_month']

df.head()

,order_id,product,quantity,age,uk_region,cpa,month,year
0,1,toothbrush,1,21,UK North East,2,Oct,2021
1,2,toys,2,35,UK South East,3,Oct,2021
2,3,toothbrush,1,24,UK North,2,Oct,2021
3,4,toothbrush,1,21,UK North East,4,Dec,2021
4,5,toothbrush,2,20,UK North,5,Oct,2021


In [10]:
df['uk_region'].value_counts()

UK South East    22
UK North East    20
UK North          8
Name: uk_region, dtype: int64

It seems that there is no data for 'UK South' regions, going forward it will be assumed that there have been no sales in this region.

In [11]:
df

,order_id,product,quantity,age,uk_region,cpa,month,year
0,1,toothbrush,1,21,UK North East,2,Oct,2021
1,2,toys,2,35,UK South East,3,Oct,2021
2,3,toothbrush,1,24,UK North,2,Oct,2021
3,4,toothbrush,1,21,UK North East,4,Dec,2021
4,5,toothbrush,2,20,UK North,5,Oct,2021
5,6,toothbrush,1,22,UK North East,2,Dec,2021
6,7,toothbrush,2,24,UK South East,1,Oct,2021
7,8,toothbrush,1,21,UK North East,6,Dec,2021
8,9,toothbrush,1,24,UK North,7,Oct,2021
9,10,toothbrush,1,21,UK North East,9,Dec,2021


After one final visual check of my dataframe, I am satisfied with the cleaned data. Side note: If this was a database much larger than 50 entries, I would consider taking value averages here, or viewing some random indexes as part of my final check.

I will now save this to a new CSV and upload it to a MySQL database before moving on to visualisations.

In [13]:
df.to_csv('clean_capstone_data.csv', index=False)